## Might wanna check before using Rull All

## Final recommendation By Title can use any csv

In [1]:
import numpy as np
import pandas as pd
import pickle
import re
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def reduce_memory(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df


In [2]:

games = reduce_memory(pd.read_csv("./games_preprocessed_with_tags_porterstemmer.csv")) ##or any csv file for recomendation by title 
games.head(5)

for i, row in games.iterrows():
    clean = re.sub('[^A-Za-z0-9]+', ' ', row["title"])
    clean = clean.lower()
    games.at[i, 'clean_title'] = clean

print(games['clean_title'].head(8))


0     prince of persia warrior within 
1          monaco what s yours is mine
2                   escape dead island
3              dungeon of the endless 
4                         metal slug 3
5                              enclave
6    hyperdimension neptunia re birth1
7                 the sum of all fears
Name: clean_title, dtype: object


### sim_matrix generation

In [3]:

count = TfidfVectorizer(stop_words='english')
count_matrixx = count.fit_transform(games["clean_title"])
print(games["clean_title"])
# count.vocabulary_
feature_names = count.get_feature_names_out()
print(feature_names)
cosine_sim_matrix = cosine_similarity(count_matrixx, dense_output=False) #gives sparse matrix
print(cosine_sim_matrix)



0        prince of persia warrior within 
1             monaco what s yours is mine
2                      escape dead island
3                 dungeon of the endless 
4                            metal slug 3
                       ...               
40479                  welcome to kowloon
40480                         taboo trial
40481                           hometopia
40482                    fading afternoon
40483                 forgive me father 2
Name: clean_title, Length: 40484, dtype: object
['00' '000' '001' ... 'zylon' 'zzzz' 'zzzzz']
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6464911 stored elements and shape (40484, 40484)>
  Coords	Values
  (0, 39632)	0.23323953061107291
  (0, 39391)	0.21404423530480962
  (0, 39254)	0.2606836025238123
  (0, 38695)	0.28370040525252027
  (0, 38663)	0.23585779522386785
  (0, 38307)	0.3777635157480767
  (0, 38001)	0.25433217017139964
  (0, 37709)	0.4119179194593477
  (0, 35642)	0.36405917209032773
  (0, 35092)	0.3218803

### Recommendation function for title

In [ ]:

n_recommendation = 20
def get_recommendations(title, cosine_sim):
	index = games[games['title'] == title].index[0]

	# if there's 2 games or more with same name (game RUSH)
	if type(index) is pd.Series:
		return []
	
	similarity_array = cosine_sim[index].toarray().flatten() #only if dense output is false i.e. sparse
	# Get the pairwise similarity scores of all games with that game
	sim_scores = list(enumerate(similarity_array))

	# Sort the games based on the similarity scores
	print(sim_scores)
	sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

	# Get the scores of the most similar games
	# (not the first one because this games as a score of 1 (perfect score) similarity
    # with itself)
	sim_scores = sim_scores[1:n_recommendation + 1]

	# Get the games indices
	movie_indices = [i[0] for i in sim_scores]

	# Return the top most similar games
	return games['title'].iloc[movie_indices].tolist()

get_recommendations("Chasm", cosine_sim_matrix)

In [ ]:
get_recommendations("Chasm", cosine_sim_matrix)